In [ ]:
import os

!pip install keras4torch
!pip install einops



os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from   copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyClassLoss,CatMap,Num_Cat
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

In [2]:
##Load DAta
X_train=pd.DataFrame(np.load('Data/higgs_small/N_train.npy')).astype(np.float64)
X_test=pd.DataFrame(np.load('Data/higgs_small/N_test.npy')).astype(np.float64).fillna(X_train.mean())
y_test=pd.DataFrame(np.load('Data/higgs_small/y_test.npy')).astype(np.int32)[0]

X_train=pd.DataFrame(np.load('Data/higgs_small/N_train.npy')).astype(np.float64).fillna(X_train.mean())
Y_train=pd.DataFrame(np.load('Data/higgs_small/y_train.npy')).astype(np.int32)[0]

X_valid=pd.DataFrame(np.load('Data/higgs_small/N_val.npy')).astype(np.float64).fillna(X_train.mean())
y_valid=pd.DataFrame(np.load('Data/higgs_small/y_val.npy')).astype(np.int32)[0]

In [3]:
#Normalise In
scalerX = StandardScaler()

scalerX.fit(X_train+np.random.normal(0,0.001,X_train.shape))

X_train = scalerX.transform(X_train).astype(np.float64)
X_valid = scalerX.transform(X_valid).astype(np.float64)
X_test  = scalerX.transform(X_test).astype(np.float64)

In [4]:
checkpoint='saved/savefileHI'

In [5]:
def True_ACC(p,t):
                return np.mean(np.where(np.round(p)==np.round(t),1,0))

In [6]:
Or_model = MainModel(
    categories        = (),               # tuple containing the number of unique values within each category
    num_continuous    = 28,
    dim               = 256  ,
    dim_out           = 2,
    depth             = 4,
    heads             = 8,
    attn_dropout      = 0.25 ,
    ff_dropout        = 0.25,
    U                 = 2,
    cases             = 16

)
no_model = Num_Cat(Or_model,num_number=28,classes=2)
model    = keras4torch.Model(no_model,).build([28])



In [ ]:

#Warm UP
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.001, total_iters=10,  verbose=False)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
callbacks=[ModelCheckpoint(checkpoint,monitor='val_acc',mode='max'),LRScheduler(sch)]
no_model.reset_Sample_size(32)

model.fit(X_train, Y_train.values,
                      epochs=10, batch_size=512,
                      validation_data=(X_test,y_test.values),
                      verbose=2,validation_batch_size=256,
                      callbacks=callbacks)

#Main Train
no_model.reset_Sample_size(32)
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5,min_lr=0.00001))
callbacks=[ModelCheckpoint(checkpoint,monitor='val_acc',mode='max'),scheduler]


curve=model.fit(X_train, Y_train.values,
        epochs=90, batch_size=512,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=256,
        callbacks=callbacks)




In [ ]:

model.load_weights(checkpoint)
no_model.reset_Sample_size(2)
def True_ACC(p,t):
                return np.mean(np.where(np.round(p)==np.round(t),1,0))

logits=0
for i in range(0,64):

        logits+=pd.DataFrame(model.predict(X_test,batch_size=512))
Test = (True_ACC(logits.idxmax(axis=1).values.reshape((-1)),y_test.values.reshape((-1))))

print('Test: ',Test)